In [43]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from time import time
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import binom
# from pingouin import pairwise_corr

from scipy.stats import barnard_exact
from scipy.stats import boschloo_exact
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency
from scipy.stats import poisson_means_test
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.proportion import proportions_chisquare

In [45]:
def g_test_adjusted(x_k, y_k, x_n, y_n):
    if x_k == 0 and y_k == 0:
        if x_n >= y_n:
            x_k += 1
            y_k += y_n / x_n
        else:
            x_k += x_n / y_n
            y_k += 1
    elif x_k == x_n and y_k == y_n:
        if x_n >= y_n:
            x_k -= 1
            y_k -= y_n / x_n
        else:
            x_k -= x_n / y_n
            y_k -= 1
    
    return chi2_contingency([[x_k, y_k],
                             [x_n - x_k, y_n - y_k]],
                            lambda_="log-likelihood")

test_list = [{
    "name": "Barnard’s exact test",
    "p_value": lambda x_k, y_k, x_n, y_n, sampling_points=32: 
                      barnard_exact([[x_k, y_k], 
                                     [x_n - x_k, y_n - y_k]],
                                    n=sampling_points,
                                    alternative="two-sided").pvalue
}, {
    "name": "Boschloo’s exact test",
    "p_value": lambda x_k, y_k, x_n, y_n, sampling_points=32: \
                      boschloo_exact([[x_k, y_k], 
                                      [x_n - x_k, y_n - y_k]],
                                     n=sampling_points,
                                     alternative="two-sided").pvalue
}, {
    "name": "Fisher’s exact test",
    "p_value": lambda x_k, y_k, x_n, y_n: \
                      fisher_exact([[x_k, y_k], 
                                    [x_n - x_k, y_n - y_k]],
                                   alternative="two-sided").pvalue
}, {
    "name": "G-test",
    "p_value": lambda x_k, y_k, x_n, y_n: g_test_adjusted(x_k, y_k, x_n, y_n).pvalue
}, {
    "name": "Poisson means test",
    "p_value": lambda x_k, y_k, x_n, y_n: \
                      poisson_means_test(x_k, x_n, y_k, y_n,
                                         alternative="two-sided").pvalue
}, {
    "name": "Z-test",
    "p_value": lambda x_k, y_k, x_n, y_n: \
                      proportions_ztest([x_k, y_k],
                                        [x_n, y_n],
                                        alternative="two-sided")[1]
}, {
    "name": "Chi-square test",
    "p_value": lambda x_k, y_k, x_n, y_n: \
                      proportions_chisquare([x_k, y_k],
                                            [x_n, y_n])[1]
}]

p_list = [
    0.0001,
    # 0.001,
    # 0.01,
    # 0.1,
    # 0.5
]

sample_size_list = [{
    "x": 10,
    "y": 10
}, {
    "x": 10,
    "y": 100
}, {
    "x": 100,
    "y": 100
# }, {
#     "x": 100,
#     "y": 1000
# }, {
#     "x": 100,
#     "y": 10000
# }, {
#     "x": 1000,
#     "y": 1000
# }, {
#     "x": 1000,
#     "y": 10000
# }, {
#     "x": 1000,
#     "y": 100000
# }, {
#     "x": 1000,
#     "y": 1000000
# }, {
#     "x": 10000,
#     "y": 10000
# }, {
#     "x": 10000,
#     "y": 100000
# }, {
#     "x": 10000,
#     "y": 1000000
# }, {
#     "x": 10000,
#     "y": 10000000
# }, {
#     "x": 10000,
#     "y": 100000000
}]

In [47]:
iter_size = 1000
default_random_state = 42
max_alpha = 1 # 0.1

for i, p in enumerate(p_list):
    for j, sample_size_couple in enumerate(sample_size_list):
        random_state = default_random_state + 2 * (i * len(sample_size_list) + j)
        x = binom.rvs(sample_size_couple["x"], p, 
                      random_state=random_state,
                      size=iter_size)
        y = binom.rvs(sample_size_couple["y"], p,
                      random_state=random_state+1,
                      size=iter_size)

        title_name = f"p = {p}, " \
                     + f"sample size x = {sample_size_couple['x']}, " \
                     + f"y = {sample_size_couple['y']}"
        print(title_name + "\n")
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=np.linspace(0, max_alpha, num=iter_size), 
                                 y=np.linspace(0, max_alpha, num=iter_size),
                                 line={"color": "black"},
                                 name="y = x",
                                 mode="lines"))
        
        for test in test_list:
            # test["total_time"] = None
            # test["mean_time"] = None
            test["p_value_list"] = []
            start_time = time()
            
            for iter_num in range(iter_size):
                p_value = test["p_value"](x[iter_num], y[iter_num],
                                          sample_size_couple["x"], sample_size_couple["y"])
                test["p_value_list"].append(p_value)
            
            end_time = time()
            # test["total_time"] = end_time - start_time
            # test["mean_time"] = test["total_time"] / iter_size
            print(f"{test['name']}: {1000 * (end_time - start_time) / iter_size:.2f} ms per test")

            p_value_ecdf = ECDF(test["p_value_list"])
            fig.add_trace(go.Scatter(x=p_value_ecdf.x[p_value_ecdf.x <= max_alpha], 
                                     y=p_value_ecdf.y[p_value_ecdf.x <= max_alpha],
                                     name=test["name"],
                                     mode="lines"))

        fig.update_layout(title=title_name,
                          xaxis_title="alpha",
                          yaxis_title="ECDF p-value",
                          legend_title="Criteria")
        fig.show()

        p_value_data = pd.DataFrame({
            test["name"]: test["p_value_list"]
            for test in test_list
        }).corr()

        fig = go.Figure()
        fig.add_trace(go.Heatmap(x=p_value_data.columns,
                                 y=p_value_data.index,
                                 z=np.array(p_value_data),
                                 text=p_value_data.values,
                                 texttemplate='%{text:.2f}'))
        fig.show()

p = 0.0001, размер выборки x = 10, y = 10
Barnard’s exact test: 14.42 ms per test
Boschloo’s exact test: 25.14 ms per test
Fisher’s exact test: 0.02 ms per test
G-test: 0.16 ms per test
Poisson means test: 0.00 ms per test
Z-test: 0.08 ms per test
Chi-square test: 0.14 ms per test


p = 0.0001, размер выборки x = 10, y = 100
Barnard’s exact test: 11.47 ms per test
Boschloo’s exact test: 28.33 ms per test
Fisher’s exact test: 0.02 ms per test
G-test: 0.18 ms per test
Poisson means test: 0.01 ms per test
Z-test: 0.08 ms per test
Chi-square test: 0.17 ms per test


p = 0.0001, размер выборки x = 100, y = 100
Barnard’s exact test: 16.31 ms per test
Boschloo’s exact test: 64.01 ms per test
Fisher’s exact test: 0.04 ms per test
G-test: 0.18 ms per test
Poisson means test: 0.01 ms per test
Z-test: 0.09 ms per test
Chi-square test: 0.14 ms per test
